In [1]:
# import sys
# sys.path.append('/home/g201381710/ExperimentsGH/DotlessArabic/')
# sys.path.append('/home/g201381710/.local/lib/python3.10/site-packages')

# sys.path

In [2]:
%cd ../../../..

/home/majed_alshaibani/Experiments/DotlessArabic


In [3]:
import tkseem as tk
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from dotless_arabic.processing import process,undot
from dotless_arabic.experiments.nlms.src import constants
from dotless_arabic.experiments.nlms.src.models import LitNeuralLanguageModel
from dotless_arabic.experiments.constants import COLLECT_DATASET_FOR_LANGUAGE_MODELLING
from dotless_arabic.tokenizers import WordTokenizer,FarasaMorphologicalTokenizer,DisjointLetterTokenizer,CharacterTokenizer
from dotless_arabic.experiments.nlms.src.utils import generate_text,get_best_checkpoint,get_tokenizer,get_dataloader,calculate_perplexity

In [4]:
# write the dataset name here :)
dataset_name = 'wikipedia'
dataset_type = 'undotted'
tokenizer_class = FarasaMorphologicalTokenizer

In [5]:
dataset = COLLECT_DATASET_FOR_LANGUAGE_MODELLING[dataset_name]()

####################################################################################################
Sample of datasets samples:
الماء مادةٌ شفافةٌ عديمة اللون والرائحة، وهو المكوّن الأساسي للجداول والبحيرات والبحار والمحيطات وكذلك للسوائل في جميع الكائنات الحيّة، وهو أكثر المركّبات الكيميائيّة انتشاراً على سطح الأرض. يتألّف جزيء الماء من ذرّة أكسجين مركزية ترتبط بها ذرّتا هيدروجين على طرفيها برابطة تساهميّة بحيث تكون صيغته الكيميائية H2O. عند الظروف القياسية من الضغط ودرجة الحرارة يكون الماء سائلاً؛ أمّا الحالة الصلبة فتتشكّل عند نقطة التجمّد، وتدعى بالجليد؛ أمّا الحالة الغازية فتتشكّل عند نقطة الغليان، وتسمّى بخار الماء.
إنّ الماء هو أساس وجود الحياة على كوكب الأرض، وهو يغطّي 71% من سطحها، وتمثّل مياه البحار والمحيطات أكبر نسبة للماء على الأرض، حيث تبلغ حوالي 96.5%. وتتوزّع النسب الباقية بين المياه الجوفيّة وبين جليد المناطق القطبيّة (1.7% لكليهما)، مع وجود نسبة صغيرة على شكل بخار ماء معلّق في الهواء على هيئة سحاب (غيوم)، وأحياناً أخرى على هيئة ضباب أو ندى، بالإضافة إلى الزخات المطري

  0%|          | 0/5609905 [00:00<?, ?it/s]

  0%|          | 0/5578668 [00:00<?, ?it/s]

####################################################################################################
Number of Samples after transformations:
11,872,890
####################################################################################################


  0%|          | 0/11872890 [00:00<?, ?it/s]

####################################################################################################
Number of Samples when considering sample with 30 tokens or more:
1,469,112
####################################################################################################


In [6]:
dataset = list(
    map(
        process,
        tqdm(dataset),
    )
)

  0%|          | 0/1469112 [00:00<?, ?it/s]

In [7]:
if dataset_type=='undotted':
    dataset = list(
        map(
            undot,
            tqdm(dataset),
        )
    )

  0%|          | 0/1469112 [00:00<?, ?it/s]

In [8]:
train_dataset, test_dataset = train_test_split(
        dataset,
        shuffle=True,
        test_size=constants.TEST_SIZE,
        random_state=constants.RANDOM_SEED,
    )

train_dataset, val_dataset = train_test_split(
    train_dataset,
    shuffle=True,
    test_size=constants.VAL_SIZE,
    random_state=constants.RANDOM_SEED,
)

In [9]:
model = LitNeuralLanguageModel.load_from_checkpoint(
        get_best_checkpoint(
            tokenizer_class=tokenizer_class,
            dataset_id=f"{dataset_type.upper()}-{dataset_name.upper()}_DATASET",
            checkpoints_base_path="/home/majed_alshaibani/Experiments/DotlessArabic/NLMs",
        )
)
model

LitNeuralLanguageModel(
  (embedding_layer): Embedding(8218, 512)
  (gru_layer): GRU(512, 512, num_layers=4, batch_first=True)
  (first_dense_layer): Linear(in_features=512, out_features=512, bias=True)
  (dropout_layer): Dropout(p=0.333, inplace=False)
  (relu): ReLU()
  (second_dense_layer): Linear(in_features=512, out_features=8218, bias=True)
)

In [10]:
tokenizer = get_tokenizer(
        train_dataset=train_dataset,
        vocab_size=model.vocab_size,
        tokenizer_class=tokenizer_class,
    )
tokenizer

[2023-03-22 13:48:57,900 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


Training FarasaMorphologicalTokenizer...


In [11]:
sequence_length=269

In [12]:
# model.learning_rate = 0.00001

In [13]:
perplexity_with_oovs = calculate_perplexity(
    lm_model=model,
    tokenizer=tokenizer,
    dataset=test_dataset,
    batch_size=128,
    sequence_length=sequence_length,
)
print(f'perplexity_with_oovs: {perplexity_with_oovs:,}')

  0%|          | 0/146912 [00:00<?, ?it/s]

  0%|          | 0/1147 [00:02<?, ?it/s]

perplexity_with_oovs: 9.593713079914767


In [14]:
perplexity_without_oovs = calculate_perplexity(
    lm_model=model,
    tokenizer=tokenizer,
    dataset=test_dataset,
    batch_size=128,
    sequence_length=sequence_length,
    ignore_oovs=True,
)
print(f'perplexity_without_oovs: {perplexity_without_oovs:,}')

  0%|          | 0/146912 [00:00<?, ?it/s]

  0%|          | 0/1147 [00:02<?, ?it/s]

perplexity_without_oovs: 9.744606261496351


In [15]:
training_perplexity = calculate_perplexity(
    lm_model=model,
    tokenizer=tokenizer,
    dataset=train_dataset,
    sequence_length=sequence_length,
    batch_size=128,
)
print(f'training_perplexity: {training_perplexity:,}')

  0%|          | 0/1256090 [00:00<?, ?it/s]

  0%|          | 0/9809 [00:04<?, ?it/s]

training_perplexity: 8.98469655311713
